# Text Preprocessing untuk Analisis Sentimen

**Tahapan Preprocessing:**
1. Load Data
2. Hapus Data Duplikat
3. Cleaning (URL, emoji, simbol, angka, username)
4. Case Folding
5. Normalisasi Kata
6. Tokenizing
7. Stopword Removal (NLTK)
8. Stemming (Sastrawi)
9. Visualisasi & Simpan Hasil
10. WordCloud Visualization
11. Bar Chart Frekuensi
12. Simpan Hasil

## 1 Import Libraries

In [49]:
# !pip install Sastrawi nltk wordcloud requests openpyxl
import pandas as pd
import re
import string
import nltk
import requests
from io import BytesIO
from collections import Counter
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\muham\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 2 Load Data

In [50]:
data = pd.read_csv('data/hasil_cleaning.csv')
print(f'Jumlah data awal: {len(data)}')
data.info()
data.head()

Jumlah data awal: 4183
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4183 entries, 0 to 4182
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   full_text      4183 non-null   object
 1   tweet_url      4183 non-null   object
 2   original_text  4183 non-null   object
 3   cleaned_text   4183 non-null   object
dtypes: object(4)
memory usage: 130.8+ KB


,full_text,tweet_url,original_text,cleaned_text
0,Ditarik karna blm lolos BPOM menurut sertifika...,https://x.com/undefined/status/200206085482159...,Ditarik karna blm lolos BPOM menurut sertifika...,ditarik karna blm lolos bpom menurut sertifika...
1,GUYSSSS... untuk kalian yang punya usaha makan...,https://x.com/undefined/status/199868753431095...,GUYSSSS... untuk kalian yang punya usaha makan...,guyssss untuk kalian yang punya usaha makananm...
2,UMKM WAJIB TAHU: Batas Sertifikasi Halal Kian ...,https://x.com/undefined/status/199839129209166...,UMKM WAJIB TAHU: Batas Sertifikasi Halal Kian ...,umkm wajib tahu batas sertifikasi halal kian d...
3,minimal lu taulah audience indo kek gimana soa...,https://x.com/undefined/status/199751040849691...,minimal lu taulah audience indo kek gimana soa...,minimal lu taulah audience indo kek gimana soa...
4,@Sentjoko @ChatGPTapp Jepang secara aktif memp...,https://x.com/undefined/status/199652135144902...,@Sentjoko @ChatGPTapp Jepang secara aktif memp...,jepang secara aktif mempromosikan makanan hala...


## 3 Pilih Kolom

In [51]:
df = pd.DataFrame(data[['cleaned_text']])
df = df.rename(columns={'cleaned_text': 'content'})
df.head()

,content
0,ditarik karna blm lolos bpom menurut sertifika...
1,guyssss untuk kalian yang punya usaha makananm...
2,umkm wajib tahu batas sertifikasi halal kian d...
3,minimal lu taulah audience indo kek gimana soa...
4,jepang secara aktif mempromosikan makanan hala...


## 4 Hapus Duplikat

In [52]:
print(f'Jumlah data sebelum hapus duplikat: {len(df)}')
duplikat = df[df.duplicated(subset='content', keep=False)]
print(f'Jumlah data duplikat: {len(duplikat)}')
df.drop_duplicates(subset='content', keep='first', inplace=True)
df = df.reset_index(drop=True)
print(f'Jumlah data setelah hapus duplikat: {len(df)}')

Jumlah data sebelum hapus duplikat: 4183
Jumlah data duplikat: 0
Jumlah data setelah hapus duplikat: 4183


## 5 Fungsi Cleaning

In [53]:
def remove_URL(tweet):
    if tweet is not None and isinstance(tweet, str):
        url = re.compile(r'https?://\S+|www\.\S+')
        return url.sub(r'', tweet)
    return tweet
def remove_emoji(tweet):
    if tweet is not None and isinstance(tweet, str):
        emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"
            u"\U0001F300-\U0001F5FF"
            u"\U0001F680-\U0001F6FF"
            u"\U0001F1E0-\U0001F1FF"
            "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', tweet)
    return tweet
def remove_symbols(tweet):
    if tweet is not None and isinstance(tweet, str):
        tweet = re.sub(r'[^a-zA-Z0-9\s]', '', tweet)
    return tweet
def remove_numbers(tweet):
    if tweet is not None and isinstance(tweet, str):
        tweet = re.sub(r'\d', '', tweet)
    return tweet
def remove_usernames(text):
    if text is not None and isinstance(text, str):
        return re.sub(r'@\w+', '', text)
    return text

## 6 Apply Cleaning

In [54]:
df['cleaning'] = df['content'].apply(lambda x: remove_URL(x))
df['cleaning'] = df['cleaning'].apply(lambda x: remove_usernames(x))
df['cleaning'] = df['cleaning'].apply(lambda x: remove_emoji(x))
df['cleaning'] = df['cleaning'].apply(lambda x: remove_symbols(x))
df['cleaning'] = df['cleaning'].apply(lambda x: remove_numbers(x))
df.head()

,content,cleaning
0,ditarik karna blm lolos bpom menurut sertifika...,ditarik karna blm lolos bpom menurut sertifika...
1,guyssss untuk kalian yang punya usaha makananm...,guyssss untuk kalian yang punya usaha makananm...
2,umkm wajib tahu batas sertifikasi halal kian d...,umkm wajib tahu batas sertifikasi halal kian d...
3,minimal lu taulah audience indo kek gimana soa...,minimal lu taulah audience indo kek gimana soa...
4,jepang secara aktif mempromosikan makanan hala...,jepang secara aktif mempromosikan makanan hala...


## 7 Case Folding

In [55]:
def case_folding(text):
    if isinstance(text, str):
        return text.lower()
    return text
df['case_folding'] = df['cleaning'].apply(case_folding)
df.head()

,content,cleaning,case_folding
0,ditarik karna blm lolos bpom menurut sertifika...,ditarik karna blm lolos bpom menurut sertifika...,ditarik karna blm lolos bpom menurut sertifika...
1,guyssss untuk kalian yang punya usaha makananm...,guyssss untuk kalian yang punya usaha makananm...,guyssss untuk kalian yang punya usaha makananm...
2,umkm wajib tahu batas sertifikasi halal kian d...,umkm wajib tahu batas sertifikasi halal kian d...,umkm wajib tahu batas sertifikasi halal kian d...
3,minimal lu taulah audience indo kek gimana soa...,minimal lu taulah audience indo kek gimana soa...,minimal lu taulah audience indo kek gimana soa...
4,jepang secara aktif mempromosikan makanan hala...,jepang secara aktif mempromosikan makanan hala...,jepang secara aktif mempromosikan makanan hala...


## 8 Normalisasi Kata

In [56]:
def replace_taboo_words(text, kamus_tidak_baku):
    if isinstance(text, str):
        words = text.split()
        replaced_words = []
        for word in words:
            if word in kamus_tidak_baku:
                baku_word = kamus_tidak_baku[word]
                if isinstance(baku_word, str) and all(char.isalpha() for char in baku_word):
                    replaced_words.append(baku_word)
                else:
                    replaced_words.append(word)
            else:
                replaced_words.append(word)
        return ' '.join(replaced_words)
    return ''
url = 'https://github.com/analysisdatasentiment/kamus_kata_baku/raw/main/kamuskatabaku.xlsx'
response = requests.get(url)
file_excel = BytesIO(response.content)
kamus_data = pd.read_excel(file_excel)
kamus_tidak_baku_dict = dict(zip(kamus_data['tidak_baku'], kamus_data['kata_baku']))
print(f'Jumlah kata dalam kamus: {len(kamus_tidak_baku_dict)}')
df['normalisasi'] = df['case_folding'].apply(lambda x: replace_taboo_words(x, kamus_tidak_baku_dict))
df.head()

Jumlah kata dalam kamus: 4347


,content,cleaning,case_folding,normalisasi
0,ditarik karna blm lolos bpom menurut sertifika...,ditarik karna blm lolos bpom menurut sertifika...,ditarik karna blm lolos bpom menurut sertifika...,ditarik karena belum lolos bpom menurut sertif...
1,guyssss untuk kalian yang punya usaha makananm...,guyssss untuk kalian yang punya usaha makananm...,guyssss untuk kalian yang punya usaha makananm...,guyssss untuk kalian yang punya usaha makananm...
2,umkm wajib tahu batas sertifikasi halal kian d...,umkm wajib tahu batas sertifikasi halal kian d...,umkm wajib tahu batas sertifikasi halal kian d...,umkm wajib tahu batas sertifikasi halal kian d...
3,minimal lu taulah audience indo kek gimana soa...,minimal lu taulah audience indo kek gimana soa...,minimal lu taulah audience indo kek gimana soa...,minimal kamu taulah audience indonesia kayak b...
4,jepang secara aktif mempromosikan makanan hala...,jepang secara aktif mempromosikan makanan hala...,jepang secara aktif mempromosikan makanan hala...,jepang secara aktif mempromosikan makanan hala...


## 9 Tokenizing

In [57]:
def tokenize(text):
    if isinstance(text, str):
        tokens = text.split()
        tokens = [t for t in tokens if len(t) > 2]
        return tokens
    return []
df['tokenize'] = df['normalisasi'].apply(tokenize)
df.head()

,content,cleaning,case_folding,normalisasi,tokenize
0,ditarik karna blm lolos bpom menurut sertifika...,ditarik karna blm lolos bpom menurut sertifika...,ditarik karna blm lolos bpom menurut sertifika...,ditarik karena belum lolos bpom menurut sertif...,"[ditarik, karena, belum, lolos, bpom, menurut,..."
1,guyssss untuk kalian yang punya usaha makananm...,guyssss untuk kalian yang punya usaha makananm...,guyssss untuk kalian yang punya usaha makananm...,guyssss untuk kalian yang punya usaha makananm...,"[guyssss, untuk, kalian, yang, punya, usaha, m..."
2,umkm wajib tahu batas sertifikasi halal kian d...,umkm wajib tahu batas sertifikasi halal kian d...,umkm wajib tahu batas sertifikasi halal kian d...,umkm wajib tahu batas sertifikasi halal kian d...,"[umkm, wajib, tahu, batas, sertifikasi, halal,..."
3,minimal lu taulah audience indo kek gimana soa...,minimal lu taulah audience indo kek gimana soa...,minimal lu taulah audience indo kek gimana soa...,minimal kamu taulah audience indonesia kayak b...,"[minimal, kamu, taulah, audience, indonesia, k..."
4,jepang secara aktif mempromosikan makanan hala...,jepang secara aktif mempromosikan makanan hala...,jepang secara aktif mempromosikan makanan hala...,jepang secara aktif mempromosikan makanan hala...,"[jepang, secara, aktif, mempromosikan, makanan..."


## 10 Stopword Removal

In [58]:
stop_words = set(stopwords.words('indonesian'))
print(f'Jumlah stopword dari NLTK: {len(stop_words)}')
def remove_stopwords(text):
    return [word for word in text if word not in stop_words]
df['stopword_removal'] = df['tokenize'].apply(lambda x: remove_stopwords(x))
df.head()

Jumlah stopword dari NLTK: 757


,content,cleaning,case_folding,normalisasi,tokenize,stopword_removal
0,ditarik karna blm lolos bpom menurut sertifika...,ditarik karna blm lolos bpom menurut sertifika...,ditarik karna blm lolos bpom menurut sertifika...,ditarik karena belum lolos bpom menurut sertif...,"[ditarik, karena, belum, lolos, bpom, menurut,...","[ditarik, lolos, bpom, sertifikasi, halal, kes..."
1,guyssss untuk kalian yang punya usaha makananm...,guyssss untuk kalian yang punya usaha makananm...,guyssss untuk kalian yang punya usaha makananm...,guyssss untuk kalian yang punya usaha makananm...,"[guyssss, untuk, kalian, yang, punya, usaha, m...","[guyssss, usaha, makananminuman, didaftarin, s..."
2,umkm wajib tahu batas sertifikasi halal kian d...,umkm wajib tahu batas sertifikasi halal kian d...,umkm wajib tahu batas sertifikasi halal kian d...,umkm wajib tahu batas sertifikasi halal kian d...,"[umkm, wajib, tahu, batas, sertifikasi, halal,...","[umkm, wajib, batas, sertifikasi, halal, kian,..."
3,minimal lu taulah audience indo kek gimana soa...,minimal lu taulah audience indo kek gimana soa...,minimal lu taulah audience indo kek gimana soa...,minimal kamu taulah audience indonesia kayak b...,"[minimal, kamu, taulah, audience, indonesia, k...","[minimal, taulah, audience, indonesia, kayak, ..."
4,jepang secara aktif mempromosikan makanan hala...,jepang secara aktif mempromosikan makanan hala...,jepang secara aktif mempromosikan makanan hala...,jepang secara aktif mempromosikan makanan hala...,"[jepang, secara, aktif, mempromosikan, makanan...","[jepang, aktif, mempromosikan, makanan, halal,..."


## 11 Stemming

In [59]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
def stem_text(text):
    return [stemmer.stem(word) for word in text]
df['steming_data'] = df['stopword_removal'].apply(lambda x: ' '.join(stem_text(x)))
df.head()

,content,cleaning,case_folding,normalisasi,tokenize,stopword_removal,steming_data
0,ditarik karna blm lolos bpom menurut sertifika...,ditarik karna blm lolos bpom menurut sertifika...,ditarik karna blm lolos bpom menurut sertifika...,ditarik karena belum lolos bpom menurut sertif...,"[ditarik, karena, belum, lolos, bpom, menurut,...","[ditarik, lolos, bpom, sertifikasi, halal, kes...",tarik lolos bpom sertifikasi halal sesuai pang...
1,guyssss untuk kalian yang punya usaha makananm...,guyssss untuk kalian yang punya usaha makananm...,guyssss untuk kalian yang punya usaha makananm...,guyssss untuk kalian yang punya usaha makananm...,"[guyssss, untuk, kalian, yang, punya, usaha, m...","[guyssss, usaha, makananminuman, didaftarin, s...",guyssss usaha makananminuman didaftarin sertif...
2,umkm wajib tahu batas sertifikasi halal kian d...,umkm wajib tahu batas sertifikasi halal kian d...,umkm wajib tahu batas sertifikasi halal kian d...,umkm wajib tahu batas sertifikasi halal kian d...,"[umkm, wajib, tahu, batas, sertifikasi, halal,...","[umkm, wajib, batas, sertifikasi, halal, kian,...",umkm wajib batas sertifikasi halal kian oktobe...
3,minimal lu taulah audience indo kek gimana soa...,minimal lu taulah audience indo kek gimana soa...,minimal lu taulah audience indo kek gimana soa...,minimal kamu taulah audience indonesia kayak b...,"[minimal, kamu, taulah, audience, indonesia, k...","[minimal, taulah, audience, indonesia, kayak, ...",minimal tau audience indonesia kayak makan ser...
4,jepang secara aktif mempromosikan makanan hala...,jepang secara aktif mempromosikan makanan hala...,jepang secara aktif mempromosikan makanan hala...,jepang secara aktif mempromosikan makanan hala...,"[jepang, secara, aktif, mempromosikan, makanan...","[jepang, aktif, mempromosikan, makanan, halal,...",jepang aktif promosi makan halal tarik wisataw...


## 12 Frekuensi Kata Awal

In [60]:
all_text = ' '.join(df['steming_data'].astype(str))
all_text = re.sub(r'[^a-zA-Z\s]', '', all_text.lower())
word_list = all_text.split()
word_freq = Counter(word_list)
sorted_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
df_freq = pd.DataFrame(sorted_freq, columns=['Kata', 'Frekuensi'])
df_freq.to_csv('data/frekuensi_kata_awal.csv', index=False)
print('✅ File frekuensi_kata_awal.csv berhasil disimpan')
df_freq.head(50)

✅ File frekuensi_kata_awal.csv berhasil disimpan


,Kata,Frekuensi
0,halal,4011
1,sertifikasi,2393
2,haram,1914
3,produk,1709
4,indonesia,1185
5,makan,1030
6,mui,934
7,ulama,824
8,majelis,732
9,bpom,492


## 13 Filter Kata

In [61]:
hapus_kata = ['ya', 'ssl', 'lr', 'pas', 'nih', 'good', 'sih']
df['steming_data'] = df['steming_data'].apply(
    lambda x: ' '.join([kata for kata in x.split() if kata not in hapus_kata])
)
kamus_tidak_baku_tambahan = {'apk': 'aplikasi', 'hp': 'telepon', 'tv': 'televisi'}
def perbaiki_kata(teks):
    kata_list = teks.split()
    hasil = [kamus_tidak_baku_tambahan.get(kata, kata) for kata in kata_list]
    return ' '.join(hasil)
df['steming_data'] = df['steming_data'].astype(str).apply(perbaiki_kata)
df.head()

,content,cleaning,case_folding,normalisasi,tokenize,stopword_removal,steming_data
0,ditarik karna blm lolos bpom menurut sertifika...,ditarik karna blm lolos bpom menurut sertifika...,ditarik karna blm lolos bpom menurut sertifika...,ditarik karena belum lolos bpom menurut sertif...,"[ditarik, karena, belum, lolos, bpom, menurut,...","[ditarik, lolos, bpom, sertifikasi, halal, kes...",tarik lolos bpom sertifikasi halal sesuai pang...
1,guyssss untuk kalian yang punya usaha makananm...,guyssss untuk kalian yang punya usaha makananm...,guyssss untuk kalian yang punya usaha makananm...,guyssss untuk kalian yang punya usaha makananm...,"[guyssss, untuk, kalian, yang, punya, usaha, m...","[guyssss, usaha, makananminuman, didaftarin, s...",guyssss usaha makananminuman didaftarin sertif...
2,umkm wajib tahu batas sertifikasi halal kian d...,umkm wajib tahu batas sertifikasi halal kian d...,umkm wajib tahu batas sertifikasi halal kian d...,umkm wajib tahu batas sertifikasi halal kian d...,"[umkm, wajib, tahu, batas, sertifikasi, halal,...","[umkm, wajib, batas, sertifikasi, halal, kian,...",umkm wajib batas sertifikasi halal kian oktobe...
3,minimal lu taulah audience indo kek gimana soa...,minimal lu taulah audience indo kek gimana soa...,minimal lu taulah audience indo kek gimana soa...,minimal kamu taulah audience indonesia kayak b...,"[minimal, kamu, taulah, audience, indonesia, k...","[minimal, taulah, audience, indonesia, kayak, ...",minimal tau audience indonesia kayak makan ser...
4,jepang secara aktif mempromosikan makanan hala...,jepang secara aktif mempromosikan makanan hala...,jepang secara aktif mempromosikan makanan hala...,jepang secara aktif mempromosikan makanan hala...,"[jepang, secara, aktif, mempromosikan, makanan...","[jepang, aktif, mempromosikan, makanan, halal,...",jepang aktif promosi makan halal tarik wisataw...


## 14 Simpan Hasil

In [62]:
df_final = df.rename(columns={'steming_data': 'preprocessed_text'})
df_final.to_csv('data/hasil_preprocessing.csv', encoding='utf8', index=False)
print('✅ File hasil_preprocessing.csv berhasil disimpan')
print(f'Jumlah data final: {len(df_final)}')
df_final.head()

✅ File hasil_preprocessing.csv berhasil disimpan
Jumlah data final: 4183


,content,cleaning,case_folding,normalisasi,tokenize,stopword_removal,preprocessed_text
0,ditarik karna blm lolos bpom menurut sertifika...,ditarik karna blm lolos bpom menurut sertifika...,ditarik karna blm lolos bpom menurut sertifika...,ditarik karena belum lolos bpom menurut sertif...,"[ditarik, karena, belum, lolos, bpom, menurut,...","[ditarik, lolos, bpom, sertifikasi, halal, kes...",tarik lolos bpom sertifikasi halal sesuai pang...
1,guyssss untuk kalian yang punya usaha makananm...,guyssss untuk kalian yang punya usaha makananm...,guyssss untuk kalian yang punya usaha makananm...,guyssss untuk kalian yang punya usaha makananm...,"[guyssss, untuk, kalian, yang, punya, usaha, m...","[guyssss, usaha, makananminuman, didaftarin, s...",guyssss usaha makananminuman didaftarin sertif...
2,umkm wajib tahu batas sertifikasi halal kian d...,umkm wajib tahu batas sertifikasi halal kian d...,umkm wajib tahu batas sertifikasi halal kian d...,umkm wajib tahu batas sertifikasi halal kian d...,"[umkm, wajib, tahu, batas, sertifikasi, halal,...","[umkm, wajib, batas, sertifikasi, halal, kian,...",umkm wajib batas sertifikasi halal kian oktobe...
3,minimal lu taulah audience indo kek gimana soa...,minimal lu taulah audience indo kek gimana soa...,minimal lu taulah audience indo kek gimana soa...,minimal kamu taulah audience indonesia kayak b...,"[minimal, kamu, taulah, audience, indonesia, k...","[minimal, taulah, audience, indonesia, kayak, ...",minimal tau audience indonesia kayak makan ser...
4,jepang secara aktif mempromosikan makanan hala...,jepang secara aktif mempromosikan makanan hala...,jepang secara aktif mempromosikan makanan hala...,jepang secara aktif mempromosikan makanan hala...,"[jepang, secara, aktif, mempromosikan, makanan...","[jepang, aktif, mempromosikan, makanan, halal,...",jepang aktif promosi makan halal tarik wisataw...


## 15 WordCloud Before vs After

In [ ]:
import numpy as np
from PIL import Image
import os

# Buat folder output
os.makedirs('output/visualizations', exist_ok=True)

# Buat stopwords tambahan
stopwords_wc = set(STOPWORDS)
stopwords_wc.update(['https', 'co', 'RT', '...', 'amp', 'lu', 'deh'])

# WordCloud Before
text_before = ' '.join(df['content'].astype(str).tolist())
wc_before = WordCloud(
    stopwords=stopwords_wc,
    background_color='white',
    max_words=500,
    width=800,
    height=400
).generate(text_before)

# Simpan ke file dulu (bypass numpy 2.x issue)
wc_before.to_file('output/visualizations/temp_before.png')
img_before = Image.open('output/visualizations/temp_before.png')

# WordCloud After
text_after = ' '.join(df_final['preprocessed_text'].astype(str).tolist())
wc_after = WordCloud(
    stopwords=stopwords_wc,
    background_color='white',
    max_words=500,
    width=800,
    height=400
).generate(text_after)

# Simpan ke file dulu (bypass numpy 2.x issue)
wc_after.to_file('output/visualizations/temp_after.png')
img_after = Image.open('output/visualizations/temp_after.png')

# Visualisasi side-by-side
plt.figure(figsize=(16, 8))

plt.subplot(1, 2, 1)
plt.imshow(img_before)
plt.axis('off')
plt.title('Before Preprocessing', fontsize=18)

plt.subplot(1, 2, 2)
plt.imshow(img_after)
plt.axis('off')
plt.title('After Preprocessing', fontsize=18)

plt.tight_layout()
plt.savefig('output/visualizations/wordcloud_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print('✅ WordCloud berhasil dibuat!')

## 16 Bar Chart Frekuensi Kata

In [ ]:
# Fungsi helper untuk membersihkan teks
def clean_text_column(series):
    cleaned = []
    for text in series.astype(str):
        text = re.sub(r"[\[\]\'\",]", "", text)
        cleaned.append(text.strip())
    return ' '.join(cleaned)

# BEFORE: kolom content
all_text_before = clean_text_column(df['content'])
words_before = all_text_before.split()
filtered_words_before = [word for word in words_before if word.lower() not in stopwords_wc]
word_counts_before = Counter(filtered_words_before)
top_words_before = word_counts_before.most_common(10)
word_before, count_before = zip(*top_words_before)

# AFTER: kolom preprocessed_text
all_text_after = clean_text_column(df_final['preprocessed_text'])
words_after = all_text_after.split()
filtered_words_after = [word for word in words_after if word.lower() not in stopwords_wc]
word_counts_after = Counter(filtered_words_after)
top_words_after = word_counts_after.most_common(10)
word_after, count_after = zip(*top_words_after)

# Plot berdampingan
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# BEFORE Plot
colors_before = plt.cm.Pastel1(range(len(word_before)))
bars1 = axes[0].bar(word_before, count_before, color=colors_before)
axes[0].set_title('Frekuensi Kata Sebelum Preprocessing', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Kata', fontsize=12)
axes[0].set_ylabel('Jumlah', fontsize=12)
axes[0].tick_params(axis='x', rotation=45)
for bar, count in zip(bars1, count_before):
    axes[0].text(bar.get_x() + bar.get_width()/2, count + 1, str(count), ha='center')

# AFTER Plot
colors_after = plt.cm.Pastel2(range(len(word_after)))
bars2 = axes[1].bar(word_after, count_after, color=colors_after)
axes[1].set_title('Frekuensi Kata Setelah Preprocessing', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Kata', fontsize=12)
axes[1].set_ylabel('Jumlah', fontsize=12)
axes[1].tick_params(axis='x', rotation=45)
for bar, count in zip(bars2, count_after):
    axes[1].text(bar.get_x() + bar.get_width()/2, count + 1, str(count), ha='center')

plt.tight_layout()
plt.savefig('output/visualizations/frekuensi_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 17 Simpan Hasil Final

In [ ]:
# Simpan hasil final
df_final.to_csv('data/Hasil_Preprocessing_Data.csv', encoding='utf8', index=False)
print('✅ File Hasil_Preprocessing_Data.csv berhasil disimpan')
print(f'\nTotal data: {len(df_final)}')
print(f'Kolom: {df_final.columns.tolist()}')
df_final.head()